In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append("lib")

import lib

import torch

In [2]:
file = pd.read_csv('burger_2d_500.csv')

In [3]:
vel_0 = file['vel_0'].to_numpy()
vel_1 = file['vel_1'].to_numpy()

In [4]:
data = np.concatenate((vel_0, vel_1))
data = data.reshape((len(data), 1))
data = np.concatenate((data,data), axis=1)
data = np.concatenate((data,data), axis=1)
data = np.concatenate((data,data), axis=1)
data = np.concatenate((data,data), axis=1)
data = data.T

u_train = np.copy(data)
u_train = torch.tensor(u_train)

In [5]:
u_train[0].shape

torch.Size([29282])

In [6]:
N = u_train.shape[1]
n = 20
encoder_dims = [100]
decoder_dims = [100]

In [7]:
nrbs = lib.nrbs.NRBS(N=N, n=n, encoder_dims=encoder_dims, decoder_dims=decoder_dims)

In [8]:
encoded = nrbs.encode(u_train)
decoded = nrbs.decode(encoded)

In [12]:
encoded.shape

torch.Size([16, 20])

In [10]:
l = torch.nn.Linear(N, 100)

In [11]:
l(u_train).shape

torch.Size([16, 100])